# 초기값이 없는 변수 : Feed 방식
 - 변수 = tf.placeholder(dtype, shape) : 
          들어오는 값에 따라 shape 자동 결정   
 - dtype : 자료형 (float32, int32, string)
 - shape : 자료구조([n] : 1차원, [r,c] : 2차원, 생략 : 공급data 결정)

In [1]:

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


# 변수 정의
a = tf.placeholder(dtype=tf.float32) # shape 생략 : 가변형
b = tf.placeholder(dtype=tf.float32) # shape 생략 : 가변형

c = tf.placeholder(dtype = tf.float32, shape=[5]) # 고정형 : 1d
d = tf.placeholder(dtype = tf.float32, shape=[None, 3]) # 고정형 : 2d [가변행, 열]

c_data = tf.random_uniform([5])

# 식 정의
mul = tf.multiply(a,b)
add = tf.add(mul,10)
c_calc = c * 0.5 # vector * scala

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
with tf.Session() as sess:
    # 변수 초기화 생략
    
    # 식 실행
    mul_re = sess.run(mul, feed_dict={a:2.5, b:2.5}) # data feed
    print("mul=", mul_re) # 6.25
    
    # 공급 data
    a_data = [1.0,2.0,3.5]
    b_data = [0.5,0.3,0.4]
    feed_data = {a:a_data, b:b_data}    
    
    mul_re2 = sess.run(mul, feed_dict=feed_data)
    print("mul2=", mul_re2) # [0.5 0.6 1.4]
    
    # 식 실행 : 식 참조
    add_re = sess.run(add,feed_dict={a:a_data, b:b_data}) # mul(a*b) + 10
    print("add_re=",add_re) # [10.5 10.6 11.4]
    
    c_data_re = sess.run(c_data) # 상수 생성
    print(c_data_re)
    print("c_calc=", sess.run(c_calc, feed_dict={c:c_data_re}))
    # [0.10019243 0.13678747 0.02176142 0.37680453 0.18358904]


mul= 6.25
mul2= [0.5 0.6 1.4]
add_re= [10.5 10.6 11.4]
[0.57793725 0.00528741 0.8762995  0.73113465 0.25871754]
c_calc= [0.28896862 0.0026437  0.43814975 0.36556733 0.12935877]


# variable Feed csv
 - csv(pandas object) -> tensorflow

In [3]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

import pandas as pd # csv file load
from sklearn.model_selection import train_test_split # data split



In [4]:

# data load
iris = pd.read_csv("C:/IITT/6_Tensorflow/data/iris.csv")
iris.info()


# 1. 공급 data 생성
cols = list(iris.columns)
x_data = iris[cols[:4]]
y_data = iris[cols[-1]]

x_data.shape # (150,4)
y_data.shape # (150, )


# 2. X,Y 변수 정의 : tensorflow 변수 정의
X = tf.placeholder(dtype=tf.float32, shape=[None,4])
Y = tf.placeholder(dtype=tf.string, shape=[None])


# 3. train/test split
x_train, x_test, y_train, y_test = train_test_split(
    x_data,y_data,test_size=0.3)
 
x_train.shape # (105,4)
x_test.shape # (45,4)



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Sepal.Length  150 non-null    float64
 1   Sepal.Width   150 non-null    float64
 2   Petal.Length  150 non-null    float64
 3   Petal.Width   150 non-null    float64
 4   Species       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


(45, 4)

In [10]:

# 4. session object : data 공급 -> 변수
with tf.Session() as sess :
    # 훈련용 data 공급
    feed_data = {X:x_train, Y:y_train}
    x_re, y_re = sess.run([X,Y],feed_dict=feed_data)
    print("x=",x_re[:10])
    print("y=",y_re[:10])
    
    # 평가용 data 공급
    test_feed_data = {X:x_test, Y:y_test}
    x_re2, y_re2 = sess.run([X,Y],feed_dict=test_feed_data)
    print("\n x=",x_re2[:10])
    print("y=",y_re2[:10])
    print(y_re2.shape) # (45,)
    print(type(y_re2)) # <class 'numpy.ndarray'>
    
    # numpy -> pandas 변경
    x_df = pd.DataFrame(x_re2, columns=['a','b','c','d'])
    print(x_df.info())
    print(x_df.mean(axis=0))
    
    y_ser = pd.Series(y_re2)
    print(y_ser.value_counts())
    


x= [[4.7 3.2 1.3 0.2]
 [5.7 2.5 5.  2. ]
 [6.2 2.2 4.5 1.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.5 1.3 0.3]
 [6.7 2.5 5.8 1.8]
 [5.4 3.  4.5 1.5]
 [5.4 3.4 1.5 0.4]
 [6.1 3.  4.6 1.4]
 [5.8 2.7 5.1 1.9]]
y= ['setosa' 'virginica' 'versicolor' 'setosa' 'setosa' 'virginica'
 'versicolor' 'setosa' 'versicolor' 'virginica']

 x= [[6.4 3.2 4.5 1.5]
 [6.4 3.1 5.5 1.8]
 [5.2 3.5 1.5 0.2]
 [5.9 3.2 4.8 1.8]
 [6.1 2.6 5.6 1.4]
 [5.6 2.8 4.9 2. ]
 [6.7 3.3 5.7 2.5]
 [7.  3.2 4.7 1.4]
 [5.9 3.  5.1 1.8]
 [5.8 4.  1.2 0.2]]
y= ['versicolor' 'virginica' 'setosa' 'versicolor' 'virginica' 'virginica'
 'virginica' 'versicolor' 'virginica' 'setosa']
(45,)
<class 'numpy.ndarray'>
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   a       45 non-null     float32
 1   b       45 non-null     float32
 2   c       45 non-null     float32
 3   d       45 non-null     float32
dtypes: float32(